# importing the required libraries

In [41]:
from pathlib import Path
from llama_index.core import VectorStoreIndex,ServiceContext,PromptTemplate,get_response_synthesizer,download_loader
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.query_engine import RetrieverQueryEngine
import torch
from transformers import BitsAndBytesConfig
from llama_index.core.evaluation import (
    FaithfulnessEvaluator,
    CorrectnessEvaluator,
    RelevancyEvaluator,
    RetrieverEvaluator,
    DatasetGenerator,
    generate_question_context_pairs
)
import nest_asyncio
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ['OPENAI_API_KEY']="abc"

# Load the data

In [2]:
PDFReader = download_loader("PDFReader")
loader = PDFReader()
documents = loader.load_data(file=Path('./Human-Nutrition-2020-Edition-1598491699.pdf'))
documents = documents[0:200] # considering 200 documents
len(documents)

/tmp/ipykernel_2647/1223085099.py:1: DeprecationWarning: Call to deprecated function (or staticmethod) download_loader. (`download_loader()` is deprecated. Please install tool using pip install directly instead.)
  PDFReader = download_loader("PDFReader")


200

# Define and create embeddings and store in Vector DB


In [3]:
embed_model = HuggingFaceEmbedding('BAAI/bge-small-en-v1.5')
vector_index = VectorStoreIndex.from_documents(documents=documents,embed_model=embed_model,show_progress=True)

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/torch/cuda/__init__.py:619: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


Parsing nodes:   0%|          | 0/200 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/200 [00:00<?, ?it/s]

# using Retrievers

In [4]:
reteriver = vector_index.as_retriever(top_k=2)
nodes = reteriver.retrieve('macronutrients functions')

for node in nodes:
    print("Node id:",node.id_)
    print("Metadata:",node.metadata)
    print("Score:",node.get_score())

Node id: cec44fb7-43ba-4373-a1d7-260224492f47
Metadata: {'page_label': '46', 'file_name': 'Human-Nutrition-2020-Edition-1598491699.pdf'}
Score: 0.7755160297111663
Node id: 7e0fcc26-7aa1-4801-9078-6b603403d9ab
Metadata: {'page_label': '49', 'file_name': 'Human-Nutrition-2020-Edition-1598491699.pdf'}
Score: 0.7651608637470783


# Build Query Engine

In [5]:
SYSTEM_PROMPT = """You are an AI assistant that answers questions in a friendly manner, based on the given source documents. Here are some rules you always follow:
- Generate human readable output, avoid creating output with gibberish text.
- Generate only the requested output, don't include any other language before or after the requested output.
- Never say thank you, that you are happy to help, that you are an AI agent, etc. Just answer directly.
- Generate professional language typically used in business documents in North America.
- Never generate offensive or foul language.
"""

query_wrapper_prompt = PromptTemplate(
    "[INST]<<SYS>>\n" + SYSTEM_PROMPT + "<</SYS>>\n\n{query_str}[/INST] "
)

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

In [9]:
# Define LLM

llm = OpenAI(temperature=0.7,model='gpt-4',verbose=True)

# Define Response Synthesizer

In [11]:
response_synthesizer = get_response_synthesizer(llm=llm,response_mode='refine')

In [14]:
query_engine = RetrieverQueryEngine(retriever=reteriver,response_synthesizer=response_synthesizer)
response = query_engine.query('macronutrients functions')
response

Response(response="Macronutrients, including carbohydrates, lipids, protein, and water, have three primary roles in the body. They supply energy, with proteins providing four kilocalories of energy per gram. They contribute to the body's structure, for instance, proteins provide structure to bones, muscles, and skin. Lastly, they regulate chemical reactions in the body, proteins are involved in most chemical reactions that occur in the body. These roles enable us to sense and react to our environment, move, eliminate waste, respire, grow, and reproduce.", source_nodes=[NodeWithScore(node=TextNode(id_='cec44fb7-43ba-4373-a1d7-260224492f47', embedding=None, metadata={'page_label': '46', 'file_name': 'Human-Nutrition-2020-Edition-1598491699.pdf'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='b0e52a8f-6478-4b92-9856-a5eaba124ee4', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'page_label': '46', 'f

# Evaluation

In [15]:
nest_asyncio.apply()
data_generator = DatasetGenerator.from_documents(documents,llm=llm)
eval_dataset = data_generator.generate_dataset_from_nodes(num=3) 

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/llama_index/core/evaluation/dataset_generation.py:212: DeprecationWarning: Call to deprecated class DatasetGenerator. (Deprecated in favor of `RagDatasetGenerator` which should be used instead.)
  return cls(
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/llama_index/core/evaluation/dataset_generation.py:309: DeprecationWarning: Call to deprecated class QueryResponseDataset. (Deprecated in favor of `LabelledRagDataset` which should be used instead.)
  return QueryResponseDataset(queries=queries, responses=responses_dict)


In [17]:
eval_questions = [example for example in eval_dataset.queries.values()]
eval_answers = [example for example in eval_dataset.responses.values()]

In [19]:
eval_query = eval_questions[0]
reference_answer = eval_answers[0]

In [24]:
query_engine = vector_index.as_query_engine(llm=llm,similarity_top_k=3)
response_vector = query_engine.query(eval_query)

service_context = ServiceContext.from_defaults(embed_model=embed_model,llm=llm)

relevancy_evaluator = RelevancyEvaluator(service_context=service_context)
eval_source_result_full = [
        relevancy_evaluator.evaluate(
            query=eval_query,
            response=response_vector.response,
            contexts=[source_node.get_content()],
        )
        for source_node in response_vector.source_nodes
    ]
eval_source_result = [
        "Pass" if result.passing else "Fail" for result in eval_source_result_full
    ]
eval_source_result

/tmp/ipykernel_2647/340935722.py:4: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(embed_model=embed_model,llm=llm)


['Fail', 'Pass', 'Fail']

In [25]:
faithfulness_evaluator = FaithfulnessEvaluator(service_context=service_context)
eval_result = faithfulness_evaluator.evaluate_response(response=response_vector) 
eval_result.score,eval_result.passing,eval_result.feedback

(1.0, True, 'YES')

In [26]:
correctness_evaluator = CorrectnessEvaluator(service_context=service_context)
eval_reference_answer = eval_answers[0]

correctness_result = correctness_evaluator.evaluate(
    query=eval_query,
    response=response_vector.response,
    reference=reference_answer,
)
correctness_result.score,correctness_result.passing,correctness_result.feedback

(5.0,
 True,
 'The generated answer is exactly correct and relevant to the user query. It provides the same information as the reference answer in a complete and clear sentence.')

In [42]:
qa_dataset = generate_question_context_pairs(nodes,llm,num_questions_per_chunk=4)

100%|██████████| 2/2 [00:10<00:00,  5.20s/it]


In [44]:
retriever_evaluator = RetrieverEvaluator.from_metric_names(
    ["mrr", "hit_rate"], retriever=reteriver
)

def display_results(name, eval_results):
    """Display results from evaluate."""

    metric_dicts = []
    for eval_result in eval_results:
        metric_dict = eval_result.metric_vals_dict
        metric_dicts.append(metric_dict)

    full_df = pd.DataFrame(metric_dicts)

    hit_rate = full_df["hit_rate"].mean()
    mrr = full_df["mrr"].mean()

    metric_df = pd.DataFrame(
        {"Retriever Name": [name], "Hit Rate": [hit_rate], "MRR": [mrr]}
    )

    return metric_df

eval_result = await retriever_evaluator.aevaluate_dataset(qa_dataset)
display_results("OpenAI Embedding Retriever", eval_result)

,Retriever Name,Hit Rate,MRR
0,OpenAI Embedding Retriever,0.875,0.8125
